<a href="https://colab.research.google.com/github/2303A510G6/ADM-LAB/blob/main/ADM_LAB_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from itertools import combinations
from collections import defaultdict

In [2]:
def get_support(transactions, itemset):
    """Calculate the support of an itemset."""
    return sum(1 for transaction in transactions if itemset.issubset(transaction)) / len(transactions)

In [3]:
def generate_frequent_itemsets(transactions, min_support):
    """Generate frequent itemsets using the Apriori algorithm."""
    itemsets = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            itemsets[frozenset([item])] += 1

    num_transactions = len(transactions)
    current_frequent_itemsets = {itemset for itemset, count in itemsets.items() if count / num_transactions >= min_support}
    all_frequent_itemsets = current_frequent_itemsets.copy()

    k = 2
    while current_frequent_itemsets:
        candidate_itemsets = set()
        for itemset1 in current_frequent_itemsets:
            for itemset2 in current_frequent_itemsets:
                union_set = itemset1 | itemset2
                if len(union_set) == k:
                    candidate_itemsets.add(union_set)

        itemsets = defaultdict(int)
        for transaction in transactions:
            for itemset in candidate_itemsets:
                if itemset.issubset(transaction):
                    itemsets[itemset] += 1

        current_frequent_itemsets = {itemset for itemset, count in itemsets.items() if count / num_transactions >= min_support}
        all_frequent_itemsets |= current_frequent_itemsets
        k += 1

    return all_frequent_itemsets

In [4]:
def generate_association_rules(frequent_itemsets, transactions, min_confidence):
    """Generate association rules from frequent itemsets."""
    rules = []
    num_transactions = len(transactions)

    for itemset in frequent_itemsets:
        if len(itemset) < 2:
            continue

        itemset_support = get_support(transactions, itemset)
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                antecedent_support = get_support(transactions, antecedent)
                confidence = itemset_support / antecedent_support if antecedent_support > 0 else 0

                if confidence >= min_confidence:
                    consequent_support = get_support(transactions, consequent)
                    lift = confidence / consequent_support if consequent_support > 0 else 0
                    rules.append((antecedent, consequent, itemset_support, confidence, lift))

    return rules

In [6]:
def main():
    transactions = [
        {'milk', 'bread', 'butter'},
        {'milk', 'bread'},
        {'milk', 'butter'},
        {'bread', 'butter'},
        {'milk', 'bread', 'butter', 'cheese'},
        {'bread', 'butter'}
    ]

    min_support = 0.5  # Minimum support threshold
    min_confidence = 0.7  # Minimum confidence threshold

    frequent_itemsets = generate_frequent_itemsets(transactions, min_support)
    rules = generate_association_rules(frequent_itemsets, transactions, min_confidence)

    print("Frequent Itemsets:")
    for itemset in frequent_itemsets:
        print(itemset)

    print("\nAssociation Rules:")
    for antecedent, consequent, support, confidence, lift in rules:
        print(f"{set(antecedent)} -> {set(consequent)} | Support: {support:.2f}, Confidence: {confidence:.2f}, Lift: {lift:.2f}")

# Change '_name_' to '__name__'
if __name__ == "__main__": # using the correct special variable __name__
    main()

Frequent Itemsets:
frozenset({'butter', 'bread'})
frozenset({'milk'})
frozenset({'milk', 'bread'})
frozenset({'butter'})
frozenset({'bread'})
frozenset({'milk', 'butter'})

Association Rules:
{'butter'} -> {'bread'} | Support: 0.67, Confidence: 0.80, Lift: 0.96
{'bread'} -> {'butter'} | Support: 0.67, Confidence: 0.80, Lift: 0.96
{'milk'} -> {'bread'} | Support: 0.50, Confidence: 0.75, Lift: 0.90
{'milk'} -> {'butter'} | Support: 0.50, Confidence: 0.75, Lift: 0.90
